The goal for running tractor is to find the ideal model of a galaxy using (a combination of) cersic, exponential, and de Vaucouleurs models. In order to do this, we need to:
- create a tractor image (which is a model of the galaxy) using the following models:
    - cersic
    - exponential
    - de Vaucouleurs
    - composites    
- take the model and compare it to the DECaLs deep imaging results of the same image

In [ ]:
import os
import copy

import fitsio
import numpy as np
import pylab as plt


from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table

import sep
import tractor

from astrometry.util.fits import fits_table
#from astrometry.util.util import wcs_pv2sip_hdr

from tractor import NullWCS
from tractor.galaxy import *
from tractor.sersic import *
from tractor.psf import *
from tractor.ellipses import *

In [ ]:
def fit_galaxies (galaxy_type, cat):
    #galaxy type: 
    # - "comp" = composite 
    # - "dev" = de Vaucouleurs 
    # - "ser" = sersic 
    # - "exp" = exponential 
    
    sources = []
    
    if galaxy_type == 'comp':
        for obj in obj_cat_bright:
            # if the object is a point source, add it as such
            if obj['point_source'] > 0.0:
                sources.append(tractor.PointSource(tractor.PixPos(obj['x'], obj['y']),
                                                   tractor.Flux(obj['flux'])) )
            else:
                sources.append(tractor.CompositeGalaxy(PixPos(obj['x'], obj['y']), 
                                                       Flux(0.4 * obj['flux']), GalaxyShape(obj['a'] * 0.8,  0.9,  obj['theta'] * 180.0 / np.pi),
                                                       Flux(0.6 * obj['flux']), GalaxyShape(obj['a'],  obj['b']/obj['a'], obj['theta'] * 180.0 / np.pi)))
    elif galaxy_type == 'dev':
        for obj in obj_cat_bright:
            # if the object is a point source, add it as such
            if obj['point_source'] > 0.0:
                sources.append(tractor.PointSource(tractor.PixPos(obj['x'], obj['y']),
                                                   tractor.Flux(obj['flux'])) )
            else:
                sources.append(tractor.DevGalaxy(tractor.PixPos(obj['x'], obj['y']),
                                         tractor.Flux(obj['flux']),
                                         GalaxyShape(obj['a'] / 2.0, 
                                                     (obj['b'] / obj['a']),
                                                     (180.0 - obj['theta'] * 180.0 / np.pi))))
    #elif galaxy_type == 'ser':
    #    for obj in obj_cat_bright:
    #        # if the object is a point source, add it as such
    #        if obj['point_source'] > 0.0:
    #            sources.append(tractor.PointSource(tractor.PixPos(obj['x'], obj['y']),
    #                                               tractor.Flux(obj['flux'])) )
    #        else:
    #            sources.append(tractor.SersicGalaxy(tractor.PixPos(obj['x'], obj['y']),
    #                                                tractor.Flux(obj['flux']),
    #                                                GalaxyShape(obj['a'] / 2.0, 
    #                                                            (obj['b'] / obj['a']),
    #                                                            (180.0 - obj['theta'] * 180.0 / np.pi))))
    elif galaxy_type == 'exp':
        for obj in obj_cat_bright:
            # if the object is a point source, add it as such
            if obj['point_source'] > 0.0:
                sources.append(tractor.PointSource(tractor.PixPos(obj['x'], obj['y']),
                                                   tractor.Flux(obj['flux'])) )
            else:
                sources.append(tractor.ExpGalaxy(tractor.PixPos(obj['x'], obj['y']),
                                                    tractor.Flux(obj['flux']),
                                                    GalaxyShape(obj['a'] / 2.0, 
                                                                (obj['b'] / obj['a']),
                                                                (180.0 - obj['theta'] * 180.0 / np.pi))))
    else: 
        raise ValueError('incorrect input for galaxy type')
        # valid types: 'comp' , 'dev' , 'exp'
                
                
    return sources         
                
    

In [ ]:
#open fits files
num = '26'
galaxy_dir = os.path.dirname(os.path.abspath(num+'_tractor_ready'))
path = os.path.join(galaxy_dir,num+'_tractor_ready')

img_data = fits.open(os.path.join(path,'img_data_crop_'+num+'.fits'))[0].data
psf_data = fits.open(os.path.join(path,'psf_data_crop_'+num+'.fits'))[0].data
psf_obj = tractor.PixelizedPSF(psf_data, Lorder=5)
sig_data = fits.open(os.path.join(path,'sig_data_crop_'+num+'.fits'))[0].data

obj_cat = Table.read(path+'/tractor_'+num+'.fits')

invvar_data = (1.0 / (sig_data ** 2.0))

w = NullWCS(pixscale=0.168)

In [ ]:
tim = tractor.Image(data=img_data,
            invvar=invvar_data,
            psf=psf_obj,
            wcs=w,
            sky=tractor.ConstantSky(0.0),
            photocal=tractor.NullPhotoCal()
            )

In [ ]:
obj_cat_bright = copy.deepcopy(obj_cat) 
obj_cat_bright.sort('flux')
obj_cat_bright.reverse()

obj_cat_bright = obj_cat_bright[:20]

In [ ]:
sources_comp = fit_galaxies('comp',obj_cat_bright)
sources_dev = fit_galaxies('dev',obj_cat_bright)
sources_exp = fit_galaxies('exp',obj_cat_bright)

In [ ]:
trac_obj_comp = Tractor([tim], sources_comp)
trac_mod_comp = trac_obj_comp.getModelImage(0, minsb=0.0)

trac_obj_dev = Tractor([tim], sources_comp)
trac_mod_dev = trac_obj_dev.getModelImage(0, minsb=0.0)

trac_obj_exp = Tractor([tim], sources_comp)
trac_mod_exp = trac_obj_exp.getModelImage(0, minsb=0.0)

In [ ]:
# composite function
plt.imshow(np.arcsinh(trac_mod_comp))
plt.title('initial model')
plt.show()
plt.imshow(np.arcsinh(img_data - trac_mod_comp))
plt.title('intial model difference between image')
plt.show()

trac_obj_comp.freezeParam('images')
trac_obj_comp.optimize_loop()

trac_mod_opt = trac_obj_comp.getModelImage(0, minsb=0., srcs=sources_comp)

plt.imshow(np.arcsinh(trac_mod_opt))
plt.title('optimized tractor model')
plt.show()

trac_mod_opt = trac_obj_comp.getModelImage(0, minsb=0., srcs=sources_comp)

plt.imshow(np.arcsinh(img_data - trac_mod_opt))
plt.title('optimized model, image difference')
plt.show()

plt.imshow(np.arcsinh(img_data))
plt.title('initial image')
plt.show()

In [ ]:
# de vac function
plt.imshow(np.arcsinh(trac_mod_dev))
plt.title('initial model')
plt.show()
plt.imshow(np.arcsinh(img_data - trac_mod_dev))
plt.title('intial model difference between image')
plt.show()

trac_obj_dev.freezeParam('images')
trac_obj_dev.optimize_loop()

trac_mod_opt = trac_obj_dev.getModelImage(0, minsb=0., srcs=sources_dev)

plt.imshow(np.arcsinh(trac_mod_opt))
plt.title('optimized tractor model')
plt.show()

trac_mod_opt = trac_obj_dev.getModelImage(0, minsb=0., srcs=sources_dev)

plt.imshow(np.arcsinh(img_data - trac_mod_opt))
plt.title('optimized model, image difference')
plt.show()

plt.imshow(np.arcsinh(img_data))
plt.title('initial image')
plt.show()

In [ ]:
# exp function
plt.imshow(np.arcsinh(trac_mod_exp))
plt.title('tractor model initial')
plt.show()
plt.imshow(np.arcsinh(img_data - trac_mod_exp))
plt.title('image - initial model difference')
plt.show()

trac_obj_exp.freezeParam('images')
trac_obj_exp.optimize_loop()

trac_mod_opt = trac_obj_exp.getModelImage(0, minsb=0., srcs=sources_exp)

plt.imshow(np.arcsinh(trac_mod_opt))
plt.title('optimized tractor model')
plt.show()

trac_mod_opt = trac_obj_exp.getModelImage(0, minsb=0., srcs=sources_exp)

plt.imshow(np.arcsinh(img_data - trac_mod_opt))
plt.title('difference between model and actual')
plt.show()

plt.imshow(np.arcsinh(img_data))
plt.title('image')
plt.show()